In [ ]:
# Installing scrapy.
!pip install scrapy

In [7]:
# Importing packages.
import requests
import time
import pandas as pd
import numpy as np
import scrapy
from scrapy.http import TextResponse
from scrapy.crawler import CrawlerProcess
import re
import os
import logging

In [8]:
#Defining base_url
base_url = "https://www.menu.am"

In [11]:
# Used the full scrapy framework to scrape the required data from menu.am. 
class MenuAmScraper(scrapy.Spider):
    name = "menu_am"
    start_urls = ["https://www.menu.am/?status=all&sort=default"]
    
    custom_settings = {
        "LOG_LEVEL": logging.WARNING,
        "FEED_FORMAT": "csv",
        "FEED_URI": "Menu_am.csv"
    }
    
    def parse(self, response):
        names = []
        opening_hours = []
        ratings_draft = []
        ratings = []
        categories = []
        urls = []

        names = response.css("div.fl.list-title>a.title::text").extract()
        opening_hours = response.css("div.new_list_time_block>span.new_list_time_block_inner::text").extract()
        blocks = response.xpath("//div[contains(@class, 'item ')]")
        categories = response.css("span.restType::text").extract()
        restaurant_url = response.css("div.fl.list-title>a::attr(href)").extract()
        urls = [base_url + i for i in restaurant_url]
  

        for i in blocks:
            ratings_draft.append(a.xpath('div[@class="new_favorites_and_rates_block"]//text()').extract())
        for j in ratings_draft:
            if(j == []):
                ratings.append(0)
                continue
            ratings.append(b[0].strip())
        for names_csv, opening_hours_csv, ratings_csv, categories_csv, urls_csv in zip(names, opening_hours, ratings, categories, urls):
            yield {'Names': names_csv, 'Opening_Hours': opening_hours_csv, 'Ratings': ratings_csv, 'Categories': categories_csv, 'Urls': urls_csv}



In [12]:
# Saving the data in a csv file.
if(os.path.isfile('Menu_am.csv') == False):
    process = CrawlerProcess()
    process.crawl(MenuAmScraper)
    process.start()

In [14]:
# Reading the csv and converting it to a DataFrame.
menu_df = pd.read_csv('Menu_am.csv')

In [16]:
# 1: Which category has the top rating?
# Grouped ratings by categories, calculated the mean, and sorted values values in 
#a descending order to get the categories with highest rating on average.
menu_df['Ratings'].groupby(menu_df['Categories']).mean().sort_values(ascending = False).head()

Categories
Լավ առաջարկ                          5.0
Եվրոպական, Սուրճ և Թեյ, Իտալական     5.0
Քաղցրավենիք, Ֆրեշ, Պաղպաղակ          5.0
Ծաղիկներ և Փուչիկներ, Քաղցրավենիք    5.0
Գաստրոնոմ, Ըմպելիքներ և Խորտիկներ    5.0
Name: Ratings, dtype: float64

In [17]:
# 2:How many close exactly at or sooner than 7pm?
# Created 2 lists. First list contains data from Opening_hours column. 
# Second list is empty. 
# Using for loop to get the required data and append it to the empty list. 
# Using .loc (to get a group of values using labels) and .isin (an equivalent method on series) 
# to see the results. 
time = menu_df['Opening_Hours']
seven_or_sooner = []
for i in time:
    if(i[8:10] != ''):
        if(int(i[8:10])<=19 and int(i[8:10])>12):
            seven_or_sooner.append(i)
menu_df['Names'].loc[menu_df['Opening_Hours'].isin(seven_or_sooner)]

12                                Լեդի Ջի Քեյք 
16                                    Էլեն Քեյք
23                          Տաք Կճուճ Ֆուդ Կորտ
24                                 ՍԻՍ Նատուրալ
34                                  Ռեդի Լանչիս
35                                      Պատառիկ
39                                 Խմորի Աշխարհ
46                      Գևորգյան Գինու Գործարան
55                                         Չոկո
59                                       Բոնիտո
61                                  Ապարան Ջուր
98                                       ԲիԷլլա
107                                   Լայֆ Ֆուդ
109                                  Փարթի Թայմ
111                                    Արտ Լանչ
122                                  Դան Դեսերտ
127                                Բագետ էնդ Կո
128                                     Ֆլորեքս
132                                          ՎՍ
153                                     Բրաբիոն
178                                     